In [1]:
import requests
import numpy as np
from scipy.stats import norm
import timeseries as ts

Generating LALR tables


In [124]:
# get
requests.get("http://localhost:5000/api/timeseries").json()

{'Payload': [], 'Status': 'OK'}

In [157]:
# taken from go_client.py
# insert via rest api
def tsmaker(m, s, j):
    "returns metadata and a time series in the shape of a jittered normal"
    meta={}
    meta['order'] = int(np.random.choice([-5, -4, -3, -2, -1, 0, 1, 2, 3, 4, 5]))
    meta['blarg'] = int(np.random.choice([1, 2]))
    t = np.arange(0.0, 1.0, 0.01)
    v = norm.pdf(t, m, s) + j*np.random.randn(100)
    return meta, ts.TimeSeries(t, v)

mus = np.random.uniform(low=0.0, high=1.0, size=50)
sigs = np.random.uniform(low=0.05, high=0.4, size=50)
jits = np.random.uniform(low=0.05, high=0.2, size=50)
for i, m, s, j in zip(range(50), mus, sigs, jits):
    meta, tsrs = tsmaker(m, s, j)
    # the primary key format is ts-1, ts-2, etc
    pk = "ts-{}".format(i)
    meta['vp'] = False # augment metadata with a boolean asking if this is a  VP.
    meta['pk'] = pk
    requests.post("http://localhost:5000/api/timeseries", json={"pk":pk, "t":list(tsrs.times()), "v":list(tsrs.values())})
    requests.post("http://localhost:5000/api/timeseries/upsert", json=meta)

In [168]:
# get ts-0
requests.get("http://localhost:5000/api/timeseries?field1=order&to1=-3&sort_by=order").json()

{'Payload': [['ts-43', {'blarg': 1, 'order': -5, 'pk': 'ts-43', 'vp': False}],
  ['ts-3', {'blarg': 2, 'order': -5, 'pk': 'ts-3', 'vp': False}],
  ['ts-15', {'blarg': 1, 'order': -5, 'pk': 'ts-15', 'vp': False}],
  ['ts-6', {'blarg': 2, 'order': -5, 'pk': 'ts-6', 'vp': False}],
  ['ts-26', {'blarg': 1, 'order': -5, 'pk': 'ts-26', 'vp': False}],
  ['ts-31', {'blarg': 2, 'order': -5, 'pk': 'ts-31', 'vp': False}],
  ['ts-13', {'blarg': 1, 'order': -5, 'pk': 'ts-13', 'vp': False}],
  ['ts-18', {'blarg': 2, 'order': -5, 'pk': 'ts-18', 'vp': False}],
  ['ts-32', {'blarg': 1, 'order': -5, 'pk': 'ts-32', 'vp': False}],
  ['ts-46', {'blarg': 2, 'order': -5, 'pk': 'ts-46', 'vp': False}],
  ['ts-41', {'blarg': 2, 'order': -5, 'pk': 'ts-41', 'vp': False}],
  ['ts-40', {'blarg': 1, 'order': -4, 'pk': 'ts-40', 'vp': False}],
  ['ts-24', {'blarg': 2, 'order': -4, 'pk': 'ts-24', 'vp': False}],
  ['ts-19', {'blarg': 1, 'order': -4, 'pk': 'ts-19', 'vp': False}],
  ['ts-29', {'blarg': 1, 'order': -4, 'pk

In [159]:
# commit to disk
requests.post("http://localhost:5000/api/commit").json()

{'Payload': None, 'Status': 'OK'}

In [169]:
# delete ts
requests.delete("http://localhost:5000/api/timeseries?pk=ts-0").json()

{'Payload': None, 'Status': 'OK'}

In [170]:
# see that it's gone
requests.get("http://localhost:5000/api/timeseries?field1=pk&value1=ts-0").json()

{'Payload': [], 'Status': 'OK'}

In [171]:
# oops, want to rollback
requests.post("http://localhost:5000/api/rollback").json()

{'Payload': None, 'Status': 'OK'}

In [172]:
# get back ts
requests.get("http://localhost:5000/api/timeseries?field1=pk&value1=ts-0").json()

{'Payload': [['ts-0', {'blarg': 2, 'order': -3, 'pk': 'ts-0', 'vp': False}]],
 'Status': 'OK'}

In [173]:
# get mean of timeseries in decreasing order
requests.get("http://localhost:5000/api/timeseries/augmented?proc=stats&target=mean&target=std&sort_by=mean&sort_by_increasing=false&limit=10").json()

{'Payload': [['ts-0', {'mean': 1.0108553206547555, 'std': 2.016907354237503}],
  ['ts-1', {'mean': 1.00788759285668, 'std': 1.2979227347056146}],
  ['ts-10', {'mean': 1.0025665349561625, 'std': 1.632372447952533}],
  ['ts-11', {'mean': 1.0005718441235674, 'std': 1.8233306932350428}],
  ['ts-12', {'mean': 0.9998054163342183, 'std': 1.4063705671469096}],
  ['ts-13', {'mean': 0.9993322612926004, 'std': 0.6434443702125051}],
  ['ts-14', {'mean': 0.9986484731341833, 'std': 0.9039230225736032}],
  ['ts-15', {'mean': 0.9978332873324416, 'std': 0.8496560375408475}],
  ['ts-16', {'mean': 0.9938921134737358, 'std': 1.181010311875164}],
  ['ts-17', {'mean': 0.9891812035178377, 'std': 1.090645518617887}]],
 'Status': 'OK'}

In [174]:
# get similarity specifying five timeseries as vantage points
requests.get("http://localhost:5000/api/timeseries/similarity?pk1=ts-18&pk2=ts-10&pk3=ts-23&pk4=ts-32&pk5=ts-15&sort_by=d_vp-1").json()

{'Payload': [['ts-18',
   {'d_vp-1': 0.0,
    'd_vp-2': 1.4142135623728878,
    'd_vp-3': 1.4142135530004034,
    'd_vp-4': 1.4046951563166905}],
  ['ts-29',
   {'d_vp-1': 1.2755339990820214,
    'd_vp-2': 1.414213562365877,
    'd_vp-3': 1.4142135433399916,
    'd_vp-4': 1.3576516290024174}],
  ['ts-48',
   {'d_vp-1': 1.3069641441658117,
    'd_vp-2': 1.414213562373025,
    'd_vp-3': 1.4142135477597109,
    'd_vp-4': 1.40382146775893}],
  ['ts-28',
   {'d_vp-1': 1.3515866035605206,
    'd_vp-2': 1.4142135623717758,
    'd_vp-3': 1.414213559726606,
    'd_vp-4': 1.3890175694271858}],
  ['ts-15',
   {'d_vp-1': 1.397297596842168,
    'd_vp-2': 1.4142135587857574,
    'd_vp-3': 1.4142135609228217,
    'd_vp-4': 1.316293866084944}],
  ['ts-14',
   {'d_vp-1': 1.3976592978671694,
    'd_vp-2': 1.414213562373095,
    'd_vp-3': 1.4142135539908958,
    'd_vp-4': 1.414026111570457}],
  ['ts-41',
   {'d_vp-1': 1.4034196085138315,
    'd_vp-2': 1.4142135254922688,
    'd_vp-3': 1.4142135619308664,

In [175]:
# delete all timeseries and commit
for i in range(50):
    pk = "ts-{}".format(i)
    requests.delete("http://localhost:5000/api/timeseries?pk={}".format(pk))
requests.post("http://localhost:5000/api/commit").json()

{'Payload': None, 'Status': 'OK'}

In [176]:
# get all
requests.get("http://localhost:5000/api/timeseries").json()

{'Payload': [], 'Status': 'OK'}